In [1]:
%pip install kaggle --upgrade
#%pip install kaggle_secrets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.model_selection import train_test_split

In [3]:
api = KaggleApi()
api.authenticate()
api.competition_download_files("titanic")

In [33]:
import shutil

shutil.move("titanic.zip", "../datasets/titanic.zip")

'../datasets/titanic.zip'

In [37]:
import zipfile

with zipfile.ZipFile('../datasets/titanic.zip', 'r') as zipref:
    zipref.extractall(path="../datasets/titanic")

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


train = pd.read_csv('../datasets/titanic/train.csv')
test = pd.read_csv('../datasets/titanic/test.csv')
data = [train,test]



In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
train.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [10]:
for dataset in data:
    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True)

In [11]:
for dataset in data:
    dataset['Fare'].fillna(dataset['Fare'].mean(), inplace=True)

In [12]:
top = 'S'
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(top)

In [13]:
# Convertendo a coluna 'Embarked' para númerico.
ports = {"S": 0, "C": 1, "Q": 2}

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

In [14]:
# Convertendo a coluna 'Sex' para númerico.
genders = {"male": 0, "female": 1}

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [15]:
# Podemos observar que os nomes estão acompanhados de títulos, vamos extrair esses títulos para saber se eles tem alguma correlação entre as variáveis.
for dataset in data:
    dataset['Title'] = dataset['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [16]:
# Iremos agrupar esses títulos nas seguintes categorias:

# Miss -> Mulheres jovens.
# Mr -> Homens casados.
# Mrs -> Mulheres casadas.
# Master -> Homens jovens.
# Rare -> Capitão,médicos,reverendos e pessoas da alta realeza.

# Definindo valores para cada título.
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Lady','the Countess','Don','Sir','Jonkheer','Capt','Col','Dr','Major','Rev','Dona'],'Rare')
    dataset['Title'] = dataset['Title'].map(titles)
    dataset['Title'] = dataset ['Title']. fillna (0) 

# Taxa de sobrevivência com base nos títulos.
train[['Title','Survived']].groupby(['Title'],as_index=False).mean()
train.drop(['Name','Ticket','Cabin'],axis=1, inplace=True)

In [17]:
# Vamos unir a coluna 'SibSp' e 'Parch' para criar uma nova coluna 'Family'.
# Tudo indica que essas duas colunas juntas nos dão o número de membros de uma mesma família.
for dataset in data:
    dataset['Family'] = dataset['SibSp'] + dataset['Parch'] + 1
    
# Média de sobrevivência com base no número de membros da família no navio. 
train[['Family','Survived']].groupby(['Family'], as_index = False).mean().sort_values(by='Survived',ascending=False)

,Family,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [18]:
# Vamos verificar se quem viajava sozinho tinha mais chance de sobreviver.
for dataset in data:
    dataset['Alone'] = dataset['Family'].apply(lambda x: x if x == 1 else 0)

# Média de sobrevivência com base no número de membros da família no navio.
train[['Alone','Survived']].groupby(['Alone'], as_index = False).mean().sort_values(by='Survived',ascending=False)

,Alone,Survived
0,0,0.505650
1,1,0.303538


In [19]:
# Substituindo os valores.
for dataset in data:
    dataset['Age'] = np.where(dataset['Age'] > 73, 73, dataset['Age'])

In [20]:
# Substituindo os valores.
for dataset in data:
    dataset['Fare'] = np.where(dataset['Fare'] > 99, 99, dataset['Fare'])

In [21]:
data[0]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family,Alone
0,1,0,3,0,22.000000,1,0,7.2500,0,1,2,0
1,2,1,1,1,38.000000,1,0,71.2833,1,3,2,0
2,3,1,3,1,26.000000,0,0,7.9250,0,2,1,1
3,4,1,1,1,35.000000,1,0,53.1000,0,3,2,0
4,5,0,3,0,35.000000,0,0,8.0500,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000,0,5,1,1
887,888,1,1,1,19.000000,0,0,30.0000,0,2,1,1
888,889,0,3,1,29.699118,1,2,23.4500,0,2,4,0
889,890,1,1,0,26.000000,0,0,30.0000,1,1,1,1


In [22]:
#for dataset in data:
    #dataset.drop(['PassengerId','Name','Ticket','Cabin'],axis=1, inplace=True)
#    dataset.drop(['PassengerId','Fare','Embarked'],axis=1, inplace=True)

In [26]:
# Separando os dados de treino e teste
X_train, X_test, y_train,y_test = train_test_split(train.drop('Survived',axis=1), train['Survived'],
test_size=0.3, random_state = 7)
X_train.shape,X_test.shape

predictors = ['Age','Sex','Pclass','Alone','Fare','Title','SibSp']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values

In [24]:
### Decision Tree ###
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

decision_tree = DecisionTreeClassifier(criterion='entropy', random_state=7)
decision_tree.fit(X_train,y_train)
prev_tree = decision_tree.predict(X_test)
result_train = decision_tree.score(X_train, y_train)
result_val = cross_val_score(decision_tree,train.drop('Survived',axis=1), train['Survived'], cv=5).mean()
print(f'training score = {result_train}, while validation score = {result_val}')

training score = 0.9842873176206509, while validation score = 0.74196221203942


In [29]:
submission = pd.DataFrame({
        'PassengerId': test['PassengerId'],
        'Survived': prev_tree})

In [31]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
